# Summary of Graphical Models and BP

## 1 Graphical models

A graphical model is a compact representation of a collection of probability distributions. It consists of a graph $G = (V, E)$, directed or undirected, where each vertex $v \in V$ is accosiated with a random variable. The edges of the graph represent statistical relationships between the variables. There are several main types of graphical models:

* Bayesian networks
* Markov random fields
* Factor graphs

### 1.1 Bayesian networks

A Bayesian network is a directed graph where vertices correspond to variables. Each random variable $x_i$ in the network has an associated conditional probability distribution or local probabilistic model. A directed edge from $x_i$ to $x_j$ represents a conditional probability of the random variable, given its parents in the graph, namely $P(x_i\ |\ x_j)$. Bayesian network defines a joint probability distribution:

$$
P(x_1, \dots, x_n) = \prod_{i=1}^{n}P(x_i\ |\ \mathbf{Par}(x_i))
\tag{1.1}
$$

**Example 1 (Bayesian network).**

![](img/1 Summary of Graphical Models and BP/1.1.png)
<center>Fig 1: A simple Bayesian network. Figure is from [2]</center>

It is clear to see, that probability distribution is as follows:

$$
P(P, T, I, X, S) = P(P)P(T)P(I|P,T)P(X|I)P(S|T)
$$

### 1.2 Markov random fields

Markov Random Fields are based on undirected graphical models. As in a Bayesian network, the nodes in the graph represent the variables. An associated probability distribution factorizes into functions, each function associated with a clique of the graph:

$$
P(x_1, ..., x_n) = Z^{-1} \prod_{C \in \mathcal{C}} \psi_C(x_C)
\tag{1.2}
$$

where Z is a constant chosen to ensure that the distribution is normalized. The set $\mathcal{C}$ is often taken to be the set of all maximal cliques of the graph. For a general undirected graph the compatibility functions $\psi_C$ need not have any obvious or direct relation to probabilities or conditional distributions defined over the graph cliques. 

A special case of Markov Random Fields is the pair wise Markov Random Field where the probability distribution factors into functions of two variables and the edges of the graph correspond to constraints between pairs of variables.

**Example 2 (pairwise MRF).**

![](img/1 Summary of Graphical Models and BP/1.2.png)
<center>Fig 2: A simple  pairwise Markov Network. Figure is from [2]</center>

It is clear to see, that probability distribution is as follows:

$$
P(P_1, P_2, P_3, P_4) = \prod_{(ij)} \psi_{(ij)}(P_i, P_j) \prod_{i=1}^{4} \psi_i(P_i)
$$

### 1.3 Factor graphs

The most general parameterization is a factor graph. A factor graph simplify displaying factorization of the probability distribution. Factor graphcs explicitly draw out the factors in a factorization of the joint probability. It is possible to convert arbitrary pairwise MRF or Bayesian network into equivalent factor graph.

**Defenition 1.** Factor graph is a pair $\mathcal{F} = (G, \{f_1, \dots, f_n\})$, where

* $G = (V, E)$ is an undirected bipartite graph such that $V = X \cup F$, where $X \cup F = \emptyset$. The nodes $X$ are variable nodes, while the nodes F are factor nodes.

* Further, $f_1, \dots ,f_n$ are positive functions and the number of functions equals the number of nodes in $F = \{F_1, \dots , F_n\}$. Each node $F_i$ is associated with the corresponding function $f_i$ and each $f_i$ is a function of the neighboring variables of $F_i$, i.e. $f_i = f_i(\mathbf{Nb}(F_i))$.

The joint probability distribution of a factor graph of N variables with M functions can be found as follows:

$$
P(x_1, \dots, x_n) = Z^{-1} \prod_{a=1}^M \psi(\{x\}_a)
\tag{1.3}
$$

The constant Z is a normalization constant. 

**Example 3 (Converting a pairwise MRF into a factor graph).**

![](img/1 Summary of Graphical Models and BP/1.3.png)
<center>Fig 3: Converting a pairwise MRF into a factor graph. First figure is from [2]</center>

The square nodes are function nodes; they connect to variables that are the arguments of those functions.

---

## 2 Inference in graphical models

Both directed and undirected graphical models represent a full joint probability distribution. It is important to solve the following computational inference problems:

* computing the marginal distribution $P(\{x\}_A)$ over a particular subset $A \in V$ of nodes. Mathematically, marginal probabilities are defined in terms of sums over all the possible states of all the other nodes in the system. For example, marginal probability of the last node can be found as follows:

$$
P(x_n) = \sum_{x_1} \dots \sum_{x_{n-1}} P(x_1, \dots, x_n)
\tag{2.1}
$$

* computing the conditional distribution $P(\{x\}_A\ |\ \{x\}_B )$, where $A \cap B = \emptyset$ and $A, B \in V$
* computing the maximum a posteriori (MAP). The task is to find the most likely assignment to the variables in $A \in V$ given the evidence $B \in V$: $\text{argmax}_{\{x\}_A} P(\{x\}_A\ |\ \{x\}_B)$
* etc

Marginal probabilities that are computed approximately are called beliefs. The belief at node $i$ is denoted by $b(x_i)$.

---

## 3 Belief propagation

Let's consider the following simple pairwise Markov Random Field.

![](img/1 Summary of Graphical Models and BP/3.1.png)
<center>Fig 4: Pairwise MRF</center>

All inference tasks can be solved by directly evaluating the corresponding sums. Let’s marginalize the joint probability to find the marginal probability at node 5, $P(x_5)$. By defenition

$$
P(x_4) = Z^{-1} \sum_{x_1, x_2, x_3, x_5, x_6, x_7} P(x_1, x_2, x_3, x_4, x_5, x_6, x_7)
\tag{3.1}
$$

Let's exploit the underlying graph structure:

$$
P(x_4) = Z^{-1} \sum_{x_1, x_2, x_3, x_5, x_6, x_7} \psi_1(x_1, x_4) \psi_2(x_2, x_4) \psi_3(x_3, x_4) \psi_4(x_4, x_5) \psi_5(x_5, x_6) \psi_6(x_5, x_7)
\tag{3.2}
$$

This next step shows the main idea of belief propagation. Because of the modular structure of the joint probability, we can pass the summations through variables it doesn’t sum over. This gives the same marginalization result, but computed much more efficiently:

$$
\begin{aligned}
P(x_4) & = Z^{-1} \sum_{x_1, x_2, x_3, x_5, x_6} \psi_1(x_1, x_4) \psi_2(x_2, x_4) \psi_3(x_3, x_4) \psi_4(x_4, x_5) \psi_5(x_5, x_6) \left[\sum_{x_7} \psi_6(x_5, x_7)\right]=\\
&= Z^{-1} \sum_{x_1, x_2, x_3, x_5, x_6} \psi_1(x_1, x_4) \psi_2(x_2, x_4) \psi_3(x_3, x_4) \psi_4(x_4, x_5) \psi_5(x_5, x_6) \mu_{7 \rightarrow 5} (x_5)=\\
&= Z^{-1} \sum_{x_1, x_2, x_3, x_5} \psi_1(x_1, x_4) \psi_2(x_2, x_4) \psi_3(x_3, x_4) \psi_4(x_4, x_5) \left[\sum_{x_6} \psi_5(x_5, x_6)\right] \mu_{7 \rightarrow 5} (x_5)=\\
&= Z^{-1} \sum_{x_1, x_2, x_3, x_5} \psi_1(x_1, x_4) \psi_2(x_2, x_4) \psi_3(x_3, x_4) \psi_4(x_4, x_5) \mu_{6 \rightarrow 5} (x_5) \mu_{7 \rightarrow 5} (x_5)=\\
&= Z^{-1} \sum_{x_1, x_2, x_3} \psi_1(x_1, x_4) \psi_2(x_2, x_4) \psi_3(x_3, x_4) \left[ \sum_{x_5} \psi_4(x_4, x_5) \mu_{6 \rightarrow 5} (x_5) \mu_{7 \rightarrow 5} (x_5) \right] =\\
&= Z^{-1} \sum_{x_1, x_2, x_3} \psi_1(x_1, x_4) \psi_2(x_2, x_4) \psi_3(x_3, x_4) \mu_{5 \rightarrow 4} (x_4)=\\
&= Z^{-1} \left[ \sum_{x_1} \psi_1(x_1, x_4) \right] \left[ \sum_{x_2} \psi_2(x_2, x_4) \right] \left[ \sum_{x_3}\psi_3(x_3, x_4) \right] \mu_{5 \rightarrow 4} (x_4)=\\
&= Z^{-1} \mu_{1 \rightarrow 4} (x_4) \mu_{2 \rightarrow 4} (x_4) \mu_{3 \rightarrow 4} (x_4) \mu_{5 \rightarrow 4} (x_4)
\end{aligned}
\tag{3.3}
$$

The normalization constant $Z$ can be determined as follows:

$$
Z = \sum_{x_4} \mu_{1 \rightarrow 4} (x_4) \mu_{2 \rightarrow 4} (x_4) \mu_{3 \rightarrow 4} (x_4) \mu_{5 \rightarrow 4} (x_4)
\tag{3.4}
$$

**Defenition 3.1.** $\mu_{i\rightarrow j}$ or so called message is a reusable partial sum for for finding the marginal probability.

**Claim 3.1.** Exact on tress.

### 3.1 BP for pairwise MRF


### 3.2 BP for Factor graphs


### 3.3 Max-product version of BP


---

## 4 Complexity

Let's return to the Eq. (3.1). Suppose, that each $x_i$ has $k$ number of statets. If we know nothing about the structure of the joint probability, evaluation of $P(x_4)$ requires $k^6$ computations. In the more general case, we need $\mathcal{O}(k^{|V|})$ summations to compute the desired marginal probability. That's why this approach becomes intractable for PGMs with many variables.

**Claim 4.1.** Every type of inference in graphical models is NP-hard. Even simplest problem of computing the distribution over a single binary variable is NP-hard.

However, factorization of the sum as we did in Eq. (3.3) reduces the total complexity to $\mathcal{O}(|V|\ k^2)$, where $|V|$ - number of nodes in the graph.

---

## 5 Loopy belief propagation


---

*Useful literature:*

1. D Koller, N Friedman,"Probabilistic Graphical Models: Principles and Techniques", 2009
3. D Koller, N Friedman, L Getoor and B Taskar "Graphical Models in a Nutshell", 2007 [[link](https://ai.stanford.edu/~koller/Papers/Koller+al:SRL07.pdf)]
3. J Yedidia, W Freeman, "Understanding belief propagation and its generalizations", 2001 [[link](http://people.csail.mit.edu/billf/publications/Understanding_Belief_Propogation.pdf)]
4. F Pernkopf, R Peharz, S Tschiatschek, "Introduction to Probabilistic Graphical Models", 2014 [[link](https://www.researchgate.net/publication/264040257_Introduction_to_Probabilistic_Graphical_Models)]
5. B Freeman and A Torralba, MIT EECS course 6.869 "Lecture 7: graphical models and belief propagation" [[link](http://helper.ipam.ucla.edu/publications/gss2013/gss2013_11344.pdf)]
6. M Wainwright and M Jordan, "Graphical Models, Exponential Families, and Variational Inference", 2008 [[link](https://people.eecs.berkeley.edu/~wainwrig/Papers/WaiJor08_FTML.pdf)]